Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
TODAY = str(cal.iloc[-1,0])
YESTERDAY = str(cal.iloc[-2,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-12-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-12-01
2024-12-10
2024-12-11


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 316
SH600570 800
SZ002459 1300
SH600104 1100
SH600588 1500
SH600233 1000
'''



REC_LIST = [
    ('GBDT0', 'EXP_BENCH', 'c745fed5a5ec439f9abeafac76e750a5'), # GBDT BENCH_NOW r0 all
    # ('GBDT1', 'EXP_BENCH', 'ffe9470bec09417fb4db91128edc0a87'), # GBDT BENCH_NOW r1 all
    ('GBDT2', 'EXP_BENCH', 'e420ff30870d4e55a1c5e0fb207fa8a0'), # GBDT BENCH_A r0 all
    # ('GBDT3', 'EXP_BENCH', '1dcc7b1ec14e46379690c4f2d6e5242d'), # GBDT BENCH_A r1 all
    ('GBDT4', 'EXP_BENCH', '8d7e72af7f4b475f9f3646021a29bb1e'), # GBDT BENCH_B r0 all
    # ('GBDT5', 'EXP_BENCH', 'd308acba0eaf453b9f5acfc309b05c46'), # GBDT BENCH_B r1 all
    ('GBDT6', 'EXP_BENCH', 'ec4284016f6d44d897969410a5ef3ec0'), # GBDT BENCH_NOW r0 csi300
    # ('GBDT7', 'EXP_BENCH', 'c9dc8e870efc459396ea77850a7bc099'), # GBDT BENCH_NOW r1 csi300
    ('GBDT8', 'EXP_BENCH', '1f1711ba5caa4fd7a298196fd6a57558'), # GBDT BENCH_A r0 csi300
    # ('GBDT9', 'EXP_BENCH', 'a79446d3724942c58ed14c59622fbe2c'), # GBDT BENCH_A r1 csi300
    ('GBDT10', 'EXP_BENCH', '74d88afd5cb3448d831402b4e7571dc6'), # GBDT BENCH_B r0 csi300
    # ('GBDT5', 'EXP_BENCH', '58317fb6231c4ba0b4d8cecc07db822d'), # GBDT BENCH_B r1 csi300
]

# EXP_NAME, rid = REC_LIST[0]
TOPK = 5
NDROP = 1
HT = 0

info = {
    'ALGO': ['GBDT'],
    'market': ['csi300_ext'], 
    # 'market': ['all'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha158"], 
    # 'feat': ["Alpha360"], 
    'label': ['r0'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[3455325:MainThread](2024-12-12 20:11:41,231) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[3455325:MainThread](2024-12-12 20:11:41,236) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[3455325:MainThread](2024-12-12 20:11:41,237) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt)
print(account)
print(norm_pos)
print(hold_lst)

2024-12-11 09:00:00
{'SH600570': {'amount': 800}, 'SZ002459': {'amount': 1300}, 'SH600104': {'amount': 1100}, 'SH600588': {'amount': 1500}, 'SH600233': {'amount': 1000}, 'cash': 316.0}
{'_settle_type': 'None', 'position': {'SH600570': {'amount': 3727, 'price': 6.6138176918029785}, 'SH600588': {'amount': 7188, 'price': 2.616530179977417}, 'SZ002459': {'amount': 17480, 'price': 1.1653757095336914}, 'SH600104': {'amount': 12461, 'price': 1.564157485961914}, 'SH600233': {'amount': 7140, 'price': 2.0420126914978027}, 'cash': 316.0, 'now_account_value': 98215.02192354202}, 'init_cash': 316.0}
['SH600570', 'SH600588', 'SZ002459', 'SH600104', 'SH600233']


Prepare dataset

In [5]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

bt_benchmark = BENCH_Step(bt_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [6]:
dataset = init_instance_by_config(benchmark.dataset_config)
bt_dataset = init_instance_by_config(bt_benchmark.dataset_config)
# dataset.prepare('test')

[3455325:MainThread](2024-12-12 20:12:28,048) INFO - qlib.timer - [log.py:127] - Time cost: 46.347s | Loading data Done
[3455325:MainThread](2024-12-12 20:12:28,052) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | DropnaLabel Done
[3455325:MainThread](2024-12-12 20:12:28,058) INFO - qlib.timer - [log.py:127] - Time cost: 0.004s | CSZScoreNorm Done
[3455325:MainThread](2024-12-12 20:12:28,059) INFO - qlib.timer - [log.py:127] - Time cost: 0.009s | fit & process data Done
[3455325:MainThread](2024-12-12 20:12:28,060) INFO - qlib.timer - [log.py:127] - Time cost: 46.360s | Init data Done
[3455325:MainThread](2024-12-12 20:13:14,353) INFO - qlib.timer - [log.py:127] - Time cost: 46.291s | Loading data Done
[3455325:MainThread](2024-12-12 20:13:14,362) INFO - qlib.timer - [log.py:127] - Time cost: 0.003s | DropnaLabel Done
[3455325:MainThread](2024-12-12 20:13:14,377) INFO - qlib.timer - [log.py:127] - Time cost: 0.014s | CSZScoreNorm Done
[3455325:MainThread](2024-12-12 20:13:14,379

In [7]:
label_df = bt_dataset.prepare("test")
label_df = pd.DataFrame(label_df.loc[:, 'LABEL0'])
label_df.columns = ['label']
# label_df.index = label_df.index.droplevel(0)
label_df

label
datetime   instrument          
2024-12-02 SH600000    0.016860
           SH600009   -0.000560
           SH600010   -0.005051
           SH600011    0.005666
           SH600015    0.030956
...                         ...
2024-12-10 SZ002841   -0.016533
           SZ002916    0.001314
           SZ002920   -0.017960
           SZ002938    0.006693
           SZ003816    0.000000

[1799 rows x 1 columns]

Load and run model

In [25]:
def simp_eval(res_df, k, t=None, day=None):
    if day:
        date = str(cal.iloc[-day,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    if t:
        top_k_per_group = top_k_per_group.tail(t)
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win

def run_model(model):
    BT_EXP_NAME = 'backtest'

    # backtest and analysis
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, bt_dataset, recorder)
        sr.generate()
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        bt_pred_df = recorder.load_object("pred.pkl")
    res_df = pd.concat([label_df, bt_pred_df], axis=1, sort=True).reindex( label_df.index)

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        # top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 10, day=day)
        # top25ic, top25win = simp_eval(res_df, 25, 5, day=day)
        top30ic, top30win = simp_eval(res_df, 30, 10, day=day)
        top40ic, top40win = simp_eval(res_df, 40, 10, day=day)
        top50ic, top50win = simp_eval(res_df, 50, 10, day=day)
        topic, topwin = simp_eval(res_df, 50, day=day)
        # return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win, top25ic, top25win, top30ic, top30win]
        return [topwin, top5win, top10win, top20win, top30win, top40win, top50win], [topic, top5ic, top10ic, top20ic, top30ic, top40ic, top50ic]
    day1metrics = eval_day(1)
    day5metrics = eval_day(5)
    day10metrics = eval_day(10)
    day20metrics = eval_day(20)
    EXP_NAME = 'realworld_position_maker'

    # backtest and analysis
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, dataset, recorder)
        sr.generate()
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        pred_df = recorder.load_object("pred.pkl")

    return day1metrics, day5metrics, day10metrics, day20metrics, pred_df

# Run

In [29]:
info_win_dict1, info_ic_dict1 = {}, {}
info_win_dict5, info_ic_dict5 = {}, {}
info_win_dict10, info_ic_dict10 = {}, {}
info_win_dict20, info_ic_dict20 = {}, {}
pred_lst = []

for NAME, EXP_NAME, rid in REC_LIST:
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=rid)
        model = recorder.load_object("trained_model")
    day1metrics, day5metrics, day10metrics, day20metrics, pred_df = run_model(model)
    info_win_dict1[NAME], info_ic_dict1[NAME] = day1metrics
    info_win_dict5[NAME], info_ic_dict5[NAME] = day5metrics
    info_win_dict10[NAME], info_ic_dict10[NAME] = day10metrics
    info_win_dict20[NAME], info_ic_dict20[NAME] = day20metrics
    # for k in [1, 5, 10, 20]:
    #     eval(f'    info_win_dict{k}[NAME], info_ic_dict{k}[NAME] = day{k}metrics')
    pred_lst.append(pred_df)

[3455325:MainThread](2024-12-12 21:08:55,650) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...


[3455325:MainThread](2024-12-12 21:08:55,660) INFO - qlib.workflow - [recorder.py:341] - Recorder 5267df7744c7486693ba0a0918d09e53 starts running under Experiment 2 ...
[3455325:MainThread](2024-12-12 21:08:55,816) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3455325:MainThread](2024-12-12 21:08:55,822) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[3455325:MainThread](2024-12-12 21:08:55,834) INFO - qlib.workflow - [recorder.py:341] - Recorder cba00443a40e4d0db347dac97e6824f1 starts running under Experiment 4 ...
[3455325:MainThread](2024-12-12 21:08:55,967) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 4
[3455325:MainThread](2024-12-12 21:08:55,977) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3455325:MainThread](2024-12-12 21:08:55,983) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...


/home/linq/finance/private_trade/mlrun/1/c745fed5a5ec439f9abeafac76e750a5/artifacts/trained_model
trained_model
'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-02 SH600000   -0.007318
           SH600009   -0.005881
           SH600010    0.016440
           SH600011   -0.012365
           SH600015    0.002524


[3455325:MainThread](2024-12-12 21:08:56,002) INFO - qlib.workflow - [recorder.py:341] - Recorder 69e48bf341d5423e85c53a030d856abc starts running under Experiment 4 ...
[3455325:MainThread](2024-12-12 21:08:56,117) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3455325:MainThread](2024-12-12 21:08:56,266) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3455325:MainThread](2024-12-12 21:08:56,283) INFO - qlib.workflow - [recorder.py:341] - Recorder 711a3745a2a9487796dc8fcb50dd55c3 starts running under Experiment 3 ...


/home/linq/finance/private_trade/mlrun/4/cba00443a40e4d0db347dac97e6824f1/artifacts/pred.pkl
pred.pkl


...

/home/linq/finance/private_trade/mlrun/1/74d88afd5cb3448d831402b4e7571dc6/artifacts/trained_model
trained_model
'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-02 SH600000   -0.038353
           SH600009    0.004608
           SH600010    0.032711
           SH600011    0.007540
           SH600015    0.040062


[3455325:MainThread](2024-12-12 21:09:00,743) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3455325:MainThread](2024-12-12 21:09:00,890) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3455325:MainThread](2024-12-12 21:09:00,907) INFO - qlib.workflow - [recorder.py:341] - Recorder fce4ec3300a644c093a09f07dfc7c0c0 starts running under Experiment 3 ...


/home/linq/finance/private_trade/mlrun/4/3c7b0406d1cd4b309cd72a1621c5d07e/artifacts/pred.pkl
pred.pkl


[3455325:MainThread](2024-12-12 21:09:01,043) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 3
[3455325:MainThread](2024-12-12 21:09:01,053) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3455325:MainThread](2024-12-12 21:09:01,059) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3455325:MainThread](2024-12-12 21:09:01,075) INFO - qlib.workflow - [recorder.py:341] - Recorder 0625fbe2a15a4c3ead42007c8067fe42 starts running under Experiment 3 ...
[3455325:MainThread](2024-12-12 21:09:01,196) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | waiting `async_log` Done


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-11 SH600000   -0.090175
           SH600009    0.015240
           SH600010   -0.217294
           SH600011   -0.026459
           SH600015   -0.023623
/home/linq/finance/private_trade/mlrun/3/fce4ec3300a644c093a09f07dfc7c0c0/artifacts/pred.pkl
pred.pkl


In [33]:
pred_df = pd.concat(pred_lst, axis=1)
pred_df.columns = name_lst = [r[0] for r in REC_LIST]
_pred_df = pred_df

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([pred_df[n]*info_ic_dict5[n][0] for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [34]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df

In [35]:
hold_lst

['SH600570', 'SH600588', 'SZ002459', 'SH600104', 'SH600233']

In [36]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
report_pos = pos_rank

In [37]:
color_lst = [n for n in report_pos.columns if n not in ['close']]
# color_lst

In [38]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SZ002459,0.042205,0.159591,0.177756,0.073762,0.117429,0.281943,0.142114,0.077879,-0.133036,15.669999
SH600233,-0.042048,-0.062771,-0.067090,-0.059841,-0.055254,-0.023180,-0.051697,0.014971,0.048415,14.579999
SH600588,-0.024862,-0.083755,-0.069144,-0.029930,-0.070510,-0.081709,-0.059985,0.023694,0.055734,12.540000
SH600570,-0.036488,-0.193811,-0.106501,-0.030736,-0.110654,-0.220868,-0.116510,0.071602,0.106090,30.820000
SH600104,-0.098783,-0.209102,-0.251170,-0.066828,-0.077364,-0.184406,-0.147942,0.070360,0.136489,17.719999


In [39]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.head(20).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top2 = report_top.head(30).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top3 = report_top.head(40).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top4 = report_top.head(50).tail(10).style.background_gradient(axis=0, subset=color_lst)

In [40]:
sty_report_top0

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SZ000100,-0.268213,-0.393989,-0.273740,-0.219655,-0.308838,-0.287249,-0.291947,0.052979,0.263318,5.120000
SH600104,-0.098783,-0.209102,-0.251170,-0.066828,-0.077364,-0.184406,-0.147942,0.070360,0.136489,17.719999
SZ002938,-0.082148,-0.128433,-0.162701,-0.108820,-0.155952,-0.215481,-0.142256,0.042602,0.132744,36.099998
SH600010,-0.114206,-0.174720,-0.160419,-0.071940,-0.108725,-0.217294,-0.141217,0.048068,0.127459,2.110000
SZ002230,-0.118473,-0.167079,-0.101355,-0.114951,-0.116194,-0.199743,-0.136299,0.035051,0.122402,53.209999
SH600029,-0.123211,-0.113736,-0.101263,-0.155237,-0.146870,-0.110030,-0.125058,0.019622,0.114715,7.270000
SH603296,-0.064012,-0.105932,-0.097825,-0.064580,-0.111084,-0.277857,-0.120215,0.072908,0.109686,61.889999
SZ002241,-0.072433,-0.107502,-0.047067,-0.107821,-0.106332,-0.264484,-0.117607,0.069420,0.106455,27.550001
SH600570,-0.036488,-0.193811,-0.106501,-0.030736,-0.110654,-0.220868,-0.116510,0.071602,0.106090,30.820000


In [41]:
sty_report_top1

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SH603259,-0.085760,-0.167108,-0.152996,-0.056161,-0.083452,-0.105896,-0.108562,0.039382,0.098888,57.690002
SH601689,-0.012611,-0.136621,-0.115431,-0.022200,-0.107787,-0.232511,-0.104527,0.073898,0.097318,52.559998
SH600606,-0.069317,-0.100671,-0.074391,-0.105406,-0.172280,-0.095525,-0.102932,0.033714,0.095840,2.590000
SH601229,-0.039402,-0.028599,-0.046017,-0.124459,-0.149395,-0.201712,-0.098264,0.064614,0.093211,8.670000
SH600030,-0.062980,-0.148752,-0.091854,-0.059596,-0.128394,-0.111820,-0.100566,0.032634,0.091856,31.339998
SZ002142,-0.080806,-0.206274,-0.091306,-0.047726,-0.079015,-0.085851,-0.098496,0.050163,0.087184,24.600000
SH601088,-0.049162,-0.059898,-0.073198,-0.058041,-0.099717,-0.207837,-0.091309,0.054540,0.084042,43.380001
SZ002180,-0.060466,-0.101027,-0.048942,-0.072012,-0.095387,-0.149803,-0.087939,0.033124,0.079558,27.309999
SH605499,-0.090686,-0.074231,-0.100901,-0.097789,-0.109987,-0.033446,-0.084507,0.025316,0.078311,242.500000


In [42]:
sty_report_top2

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SH601059,-0.040330,-0.064179,-0.085805,-0.081554,-0.097537,-0.104545,-0.078992,0.021463,0.074822,15.960000
SH600415,-0.047884,-0.064635,-0.105590,-0.100343,-0.080977,-0.046780,-0.074368,0.023275,0.071428,15.130000
SH601066,-0.032904,-0.110934,-0.051787,-0.051144,-0.104327,-0.110909,-0.077001,0.032395,0.070773,27.700001
SZ002812,-0.039665,-0.086512,-0.047739,-0.070246,-0.080928,-0.111601,-0.072782,0.024125,0.067033,36.770000
SH600115,-0.054861,-0.043159,-0.085122,-0.065234,-0.067350,-0.104534,-0.070043,0.020003,0.065234,4.330000
SH601788,-0.070839,-0.112513,-0.051351,-0.016325,-0.062963,-0.125221,-0.073202,0.036684,0.063445,20.240002
SZ002236,-0.033583,-0.064866,-0.039490,-0.036005,-0.110148,-0.106586,-0.065113,0.032265,0.059825,16.940001
SH600999,-0.031529,-0.095715,-0.047141,-0.033607,-0.079947,-0.094137,-0.063679,0.027172,0.058007,19.910002
SH600050,-0.039355,-0.066381,-0.045966,-0.023298,-0.067555,-0.129665,-0.062036,0.033903,0.055821,5.290000


In [43]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

dfi.export(infodf, './tmp/ensemble_info.png',table_conversion='matplotlib')

# dfi.export(sty_report_info5, './tmp/ensemble_performance5.png',table_conversion='matplotlib')
# dfi.export(sty_report_info10, './tmp/ensemble_performance10.png',table_conversion='matplotlib')
# dfi.export(sty_report_info20, './tmp/ensemble_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_top2.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_top3.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_top4.png',table_conversion='matplotlib')
dfi.export(sty_report_pos, './tmp/ensemble_watch.png',table_conversion='matplotlib')

In [ ]:

for k in [1,5,10,20]:
    sty_report_win_info = eval(f"pd.DataFrame(info_win_dict{k}, index=['TOP50WIN', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50'])").style.background_gradient(axis=0)
    sty_report_ic_info = eval(f"pd.DataFrame(info_ic_dict{k}, index=['TOP50IC', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50'])").style.background_gradient(axis=0)
    dfi.export(sty_report_win_info, f'./tmp/ensemble_win{k}.png',table_conversion='matplotlib')
    dfi.export(sty_report_ic_info, f'./tmp/ensemble_ic{k}.png',table_conversion='matplotlib')